<style>
.container { width: 80% !important; margin: auto; }
div.cell { padding: 10px 15% 10px 15%; }
</style>

## DTSC 2301 Spring 2026 Homework #3

Turn in your assignment via Gradescope

Due 2/8/26, 11:59pm


#### Question 1

For this questions, we will analyze some data regarding players from the National Hockey League for the 2019 season and the 2020 season.  
Here's the link to that data
[<https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/NHLShootingPct.csv>].  
Note that the data is only for even-strength situations when each team has the same number of skaters on the ice.
  
From these data run a regression predicting the number of shots a player got in 2020, *Shots2020*, from the number of shots that they took in 2019, *Shots2019*.  Find the $r$, $r^2$ and the $RMSE$.  Interpret each of those.


#### Question 2
From the regression you ran in Question 1, determine if there are any outliers, leverage or influential points.  Explain your reasoning for your answer to each type of unusual observation.

#### Question 3
You want to choose the best feature to predict *Goals2020* from among *Goals2019*, *Sh2019*, and *Shots2019*.  How would you decide which of these is the best predictor of *Goals2020*?  Explain clearly why you would choose that methodology.  *Sh2019* is the shooting percentage or the percent of shots that were goals.

#### Question 4
Now fit a multiple regression to predict *Goals2020* with all three of the predictors from Question 3.  Is there evidence of multicollinearity?  Explain how you know.

#### Question 5

**Be sure that your response is fully readable in your submission**

Sports executives use predictive modeling to maximize success of the team with minimal investment. Given the transparency and abundance of performance data, write a short 80-120 word paragraph explaining how bias can be introduced in sports models, and how introducing bias can affect conclusions?

#### Question 6

**Be sure that your response is fully readable in your submission**

What strategies can be employed to mitigate the biases you discussed in Question 5?"

### Q1
Here's the link to that data [https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/NHLShootingPct.csv].

Note that the data is only for even-strength situations when each team has the same number of skaters on the ice.

From these data run a regression predicting the number of shots a player got in 2020, Shots2020, from the number of shots that they took in 2019, Shots2019. Find the  r ,  r2  and the  RMSE . Interpret each of those.



In [1]:
# Load Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


df_nhl_shooting = pd.read_csv('https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/NHLShootingPct.csv')
df_nhl_shooting.head()

,ID,Player,Pos,Goals2019,Shots2019,Sh2019,Goals2020,Shots2020,Sh2020,id
0,8479318,Auston Matthews,C,31,199,0.155779,28,159,0.176101,1
1,8478402,Connor McDavid,C,21,143,0.146853,19,130,0.146154,2
2,8478420,Mikko Rantanen,R,9,69,0.130435,18,107,0.168224,3
3,8478493,Joel Eriksson Ek,C,7,92,0.076087,17,108,0.157407,4
4,8478483,Mitchell Marner,R,7,97,0.072165,17,103,0.165049,5


In [3]:
#Q1 Find r, r^2 and RMSE

X = sm.add_constant(df_nhl_shooting['Shots2019'])
y = df_nhl_shooting['Shots2020']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
residuals = y - predictions
rmse = np.sqrt(np.mean(residuals**2))
r_squared = model.rsquared
r = np.sqrt(r_squared)
print(f"r: {r}")
print(f"r^2: {r_squared}")
print(f"RMSE: {rmse}")

r: 0.6328298380606219
r^2: 0.4004736039398329
RMSE: 21.21281626393402


### Q1 Interpretation

*   r = 0.633 indicates a moderately strong positive relationship between predictor shots in 2019 and response shots in 2020
*   About 40% of the variation in Shots2020 can be explained by Shots2019
*   On average, the predicted number of shots in 2020 differs from the actual number by about 21.21 shots





### Q2
From the regression you ran in Question 1, determine if there are any outliers, leverage or influential points. Explain your reasoning for your answer to each type of unusual observation.

In [15]:
# Determine influence, high leverage, influential points

influence = model.get_influence()
standardized_residuals = influence.resid_studentized_internal
leverage = influence.hat_matrix_diag
cooks_d = influence.cooks_distance[0]
outliers = np.where(np.abs(standardized_residuals) > 2)[0]
high_leverage = np.where(leverage > 2 * (X.shape[1] / X.shape[0]))[0]
influential_points = np.where(cooks_d > 4 / X.shape[0])[0]
print(f"Outliers (standardized residuals > 2): {outliers}")
print(f"High leverage points: {high_leverage}")
print(f"Influential points (Cook's distance > 4/n): {influential_points}")
print("Number of outliers:", len(outliers))
print("Number of high leverage points:", len(high_leverage))
print("Number of influential points:", len(influential_points))

Outliers (standardized residuals > 2): [  2  10  20  22  26  31  40  56  63  73 115 150 246 286 316 319 330 331
 340]
High leverage points: [  0  17  19  33  47  57  71 117]
Influential points (Cook's distance > 4/n): [  0   1  10  20  22  26  31  33  37  47  63  71 107 246 286 330]
Number of outliers: 19
Number of high leverage points: 8
Number of influential points: 16


Based on standardized residuals (|r| > 2), 19 observations were identified as potential outliers, indicating large prediction errors. Using the leverage threshold
2
𝑝
/
𝑛
2p/n, 8 observations were classified as high leverage, meaning they have unusual predictor values. According to Cook’s distance (> 4/n), 16 observations were found to be influential, suggesting they may substantially affect the regression coefficients and should be further examined

## Q3
You want to choose the best feature to predict Goals2020 from among Goals2019, Sh2019, and Shots2019. How would you decide which of these is the best predictor of Goals2020? Explain clearly why you would choose that methodology. Sh2019 is the shooting percentage or the percent of shots that were goals.

In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Data
df = df_nhl_shooting[["Goals2020", "Goals2019", "Sh2019", "Shots2019"]].dropna()
y = df["Goals2020"].values

predictors = ["Goals2019", "Sh2019", "Shots2019"]
results = []

# Fit 3 separate simple linear regression models
for feat in predictors:
    X = df[[feat]].values
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)

    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))

    results.append([feat, r2, rmse])

# Compare using R^2 and RMSE
results_df = pd.DataFrame(results, columns=["Predictor", "R2", "RMSE"]) \
               .sort_values(by=["R2", "RMSE"], ascending=[False, True])

print(results_df)

best = results_df.iloc[0]["Predictor"]
print("\nBest single predictor (highest R2, lowest RMSE):", best)


   Predictor        R2      RMSE
0  Goals2019  0.225064  3.643534
2  Shots2019  0.210585  3.677415
1     Sh2019  0.039033  4.057367

Best single predictor (highest R2, lowest RMSE): Goals2019


Expalination- I fit three separate simple linear regression models using Goals2019, Shots2019, and Sh2019 to predict Goals2020. I compared the models using R² and RMSE. Since Goals2019 had the highest R² and the lowest RMSE, it is the best single predictor because it explains the most variation and produces the most accurate predictions.

## Q 4
Now fit a multiple regression to predict Goals2020 with all three of the predictors from Question 3. Is there evidence of multicollinearity? Explain how you know.

In [16]:
# 1. Multiple regression model

y = df_nhl_shooting["Goals2020"]
X = df_nhl_shooting[["Goals2019", "Sh2019", "Shots2019"]]
X_const = sm.add_constant(X)

model_multi = sm.OLS(y, X_const).fit()
print(model_multi.summary())


# 2. Quick correlation check (collinearity)

print("\nCorrelation among predictors:")
print(X.corr())


# 3. VIF (main multicollinearity check)

vif_df = pd.DataFrame()
vif_df["feature"] = X_const.columns
vif_df["VIF"] = [variance_inflation_factor(X_const.values, i)
                 for i in range(X_const.shape[1])]
print("\nVIF table:")
print(vif_df)


# 4. Condition number (another indicator)
cond_num = np.linalg.cond(X_const.values)
print("\nCondition number:", cond_num)


                            OLS Regression Results                            
Dep. Variable:              Goals2020   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     37.90
Date:                Thu, 19 Feb 2026   Prob (F-statistic):           3.53e-21
Time:                        14:56:15   Log-Likelihood:                -935.59
No. Observations:                 347   AIC:                             1879.
Df Residuals:                     343   BIC:                             1895.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.7044      1.499      2.471      0.0

Explaination-
There is evidence of multicollinearity in the model.
The correlation matrix shows a strong correlation between Goals2019 and Shots2019 (r = 0.77), indicating they are highly related.

The VIF values are high:

Goals2019 = 17.47

Shots2019 = 10.01

Sh2019 = 7.21
Since VIF > 5 (and especially >10) suggests multicollinearity, this is strong evidence.
The condition number is 7058, which is very large (values >30–100 indicate potential problems)

## Q5
Be sure that your response is fully readable in your submission

Sports executives use predictive modeling to maximize success of the team with minimal investment. Given the transparency and abundance of performance data, write a short 80-120 word paragraph explaining how bias can be introduced in sports models, and how introducing bias can affect conclusions?

### Answer
Bias can be introduced into sports predictive models in several ways. Selection bias occurs when only certain players, seasons, or teams are included, limiting how well results generalize. Survivorship bias arises when only successful players remain in the dataset, overstating consistency and talent. Measurement bias can occur if statistics like shots or goals are recorded inconsistently. Omitted variable bias may result when important factors such as injuries, playing time, or team strategy are excluded from the model. These biases can distort predictions, leading executives to overestimate performance, underestimate risk, and make inefficient roster or financial decisions that hurt long-term team success.

## Q6
**Be sure that your response is fully readable in your submission**

What strategies can be employed to mitigate the biases you discussed in Question 5?


### Answer

1. Contextual variables (ice time, position, team strength) → reduces omitted variable bias

2. Using multiple seasons of data → reduces short-term randomness and historical bias

3. Cross-validation → prevents overfitting and improves generalization

4. Comparing multiple models/metrics → reduces reliance on a single biased metric

5. Transparency in assumptions → helps identify and correct hidden bias

